In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd clean/RecSys/code

/teamspace/studios/this_studio/clean/RecSys/code


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from dataclasses import dataclass

@dataclass
class ArgsVal:
    dataset: str = "demo"
    T: int = 4  # number of previous articles to consider in the prompt
    datafraction: float = 1.0  # how much of entire dataset to use
    model: str = "CG"  # just placeholder conditional generation
    backbone: str = "google/mt5-small"
    tokenizer: str = "google/mt5-small"
    evaltrain: bool = False  # allows for evaluation on training set

args = ArgsVal()

In [4]:
from data.collators import CollatorValidation
from transformers import AutoTokenizer
from data.dataloader import EkstraBladetDataset
from torch.utils.data import DataLoader
from utils.prompt_templates import create_prompt_titles
from utils.metrics import MetricsEvaluator

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
collator = CollatorValidation(tokenizer)
data = EkstraBladetDataset(args, create_prompt_titles, split="validation")
dl = DataLoader(data, batch_size=1, collate_fn=collator, num_workers=8, shuffle=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
from collections import defaultdict, Counter
import numpy as np
from tqdm import tqdm

In [7]:
metrics_evaluator = MetricsEvaluator()
results = defaultdict(int)
total = 0
for ex in tqdm(iter(dl)):
    pubtimes = ex["publish_time"]
    last_clicked_article = pubtimes[3]
    inview_articles_times = pubtimes[4:]

    # Calculate the absolute difference in seconds between each timestamp and the reference timestamp
    differences = [abs((ts - last_clicked_article).total_seconds()) for ts in inview_articles_times]

    # Normalize the differences to convert them into scores (higher score for closer timestamps)
    # We'll use an inverse proportionality for scoring
    max_diff = max(differences)
    scores = [(max_diff - diff) for diff in differences]

    # Normalize scores to be between 0 and 1 (optional, for better interpretability)
    min_score, max_score = min(scores), max(scores)
    scores = [(score - min_score) / (max_score - min_score) for score in scores]

    metrics = metrics_evaluator.compute_metrics({
        'scores': np.array(scores),
        'labels': np.array(ex["targets"]),
        'categories': np.array(ex["categories"]),
    })

    # Sum the metrics
    for key in metrics:
        results[key] += metrics[key]
    total += 1

# Take the average of the metrics over the dataset
for key in results:
    results[key] /= total
results

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
  0%|          | 0/25356 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length

defaultdict(int,
            {'AUC': 0.6916642188873707,
             'ndcg': 0.5322142751724712,
             'ndcg@5': 0.44624063306448813,
             'mrr': 0.38956835789108085,
             'mrr@5': 0.35576786559392687,
             'precision@1': 0.1690724088973024,
             'recall@5': 0.7237241678498186,
             'hit_ratio@5': 0.7242467266130305,
             'diversity@1': 0.49451806278592836,
             'diversity@2': 0.4913827102066572,
             'diversity@3': 0.48883893358574376,
             'diversity@4': 0.4880205868433507,
             'diversity@5': 0.48997475942578106,
             'intra_list_diversity@1': 1.0,
             'intra_list_diversity@2': 0.8861019088184257,
             'intra_list_diversity@3': 0.8097623179260819,
             'intra_list_diversity@4': 0.745149077141505,
             'intra_list_diversity@5': 0.6928616501024476})

In [75]:
import random

In [79]:
metrics_evaluator = MetricsEvaluator()
results = defaultdict(int)
total = 0
np.random.seed(42)
random.seed(42)

for ex in tqdm(iter(dl)):
    pubtimes = ex["publish_time"]
    random_scores = np.random.random(len(pubtimes)-4)

    metrics = metrics_evaluator.compute_metrics({
        'scores': np.array(random_scores),
        'labels': np.array(ex["targets"]),
        'categories': np.array(ex["categories"]),
    })

    # Sum the metrics
    for key in metrics:
        results[key] += metrics[key]
    total += 1

# Take the average of the metrics over the dataset
for key in results:
    results[key] /= total

results

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
  1%|        

100%|██████████| 25356/25356 [00:50<00:00, 501.71it/s]


In [85]:
metrics_evaluator = MetricsEvaluator()
results = defaultdict(int)
total = 0
for ex in tqdm(iter(dl)):
    categories = ex["categories"]
    last_clicked_article = categories[:4]
    inview_articles_cats = categories[4:]

    category_counts = Counter(last_clicked_article)
    m = max(category_counts.values())
    scores = [category_counts[category]/m for category in inview_articles_cats]

    metrics = metrics_evaluator.compute_metrics({
        'scores': np.array(scores),
        'labels': np.array(ex["targets"]),
        'categories': np.array(ex["categories"]),
    })

    # Sum the metrics
    for key in metrics:
        results[key] += metrics[key]
    total += 1

# Take the average of the metrics over the dataset
for key in results:
    results[key] /= total
results

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
  0%|          | 0/25356 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length

100%|██████████| 25356/25356 [00:50<00:00, 497.24it/s]


defaultdict(int,
            {'AUC': 0.5968676248598606,
             'ndcg': 0.49218359095923137,
             'ndcg@5': 0.37856899367455715,
             'mrr': 0.3427683923720312,
             'mrr@5': 0.30025569227530546,
             'precision@1': 0.14434453383814483,
             'recall@5': 0.6199584582215912,
             'hit_ratio@5': 0.6204843035179051,
             'diversity@1': 0.02539832781195772,
             'diversity@2': 0.058625177472787504,
             'diversity@3': 0.10931009097123734,
             'diversity@4': 0.1708274175737498,
             'diversity@5': 0.2369774412367776,
             'intra_list_diversity@1': 1.0,
             'intra_list_diversity@2': 0.6913748225272125,
             'intra_list_diversity@3': 0.5985434085292004,
             'intra_list_diversity@4': 0.5592364726297523,
             'intra_list_diversity@5': 0.5407556396908079})